In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, GeoJson, GeoJsonPopup
from folium.plugins import HeatMap, MarkerCluster

Importando os dados de focos de queimadas do INPE

In [ ]:
!pip install -q gdown
!gdown https://drive.google.com/uc?id=1IFbCtkYOtDJwxCwAknQIOPwwptw3CO6F -O focos_br_todos-sats_2024.csv
!gdown https://drive.google.com/uc?id=1ocbEh-ZOFAZGrsfUMZWPt9SQjEhrW4m4 -O biomas_brasil.zip
!gdown https://drive.google.com/uc?id=1U27fhOaGLOiEa_N76is59PCUswC65m2p -O ufs.zip
!unzip biomas_brasil.zip -d biomas
!unzip ufs.zip -d ufs

Downloading...
From (original): https://drive.google.com/uc?id=1IFbCtkYOtDJwxCwAknQIOPwwptw3CO6F
From (redirected): https://drive.google.com/uc?id=1IFbCtkYOtDJwxCwAknQIOPwwptw3CO6F&confirm=t&uuid=35ab8ceb-024a-443f-8c84-68aa74c3227e
To: /content/focos_br_todos-sats_2024.csv
100% 905M/905M [00:06<00:00, 132MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ocbEh-ZOFAZGrsfUMZWPt9SQjEhrW4m4
To: /content/biomas_brasil.zip
100% 15.1M/15.1M [00:00<00:00, 44.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1U27fhOaGLOiEa_N76is59PCUswC65m2p
To: /content/ufs.zip
100% 13.8M/13.8M [00:00<00:00, 82.8MB/s]
Archive:  biomas_brasil.zip
 extracting: biomas/lm_bioma_250.CPG  
  inflating: biomas/lm_bioma_250.dbf  
  inflating: biomas/lm_bioma_250.prj  
  inflating: biomas/lm_bioma_250.sbn  
  inflating: biomas/lm_bioma_250.sbx  
  inflating: biomas/lm_bioma_250.shp  
  inflating: biomas/lm_bioma_250.shp.xml  
  inflating: biomas/lm_bioma_250.shx  
Archive:  ufs.zip
 extracting: ufs/BR

In [ ]:
dados = pd.read_csv('focos_br_todos-sats_2024.csv')

Aqui, utilizei a função value_counts para ver quais satélites foram utilizados, decidi utilizar o satélite NOAA-21 como base das análises, para evitar pontos duplicados.

In [ ]:
dados.satelite.value_counts()
dados_noa21 = dados.loc[dados.satelite == 'NOAA-21']

Visualizando os dados que serão trabalhados.

In [ ]:
dados_gdf = gpd.GeoDataFrame(dados_noa21, geometry=gpd.points_from_xy(dados_noa21.longitude, dados_noa21.latitude))

dados_gdf.crs = {'init' : 'epsg:4326'}
dados_gdf.head()

/usr/local/lib/python3.11/dist-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,latitude,longitude,data_pas,satelite,pais,estado,municipio,bioma,numero_dias_sem_chuva,precipitacao,risco_fogo,id_area_industrial,frp,geometry
103966,-6.10801,-35.36419,2024-01-17 03:57:00,NOAA-21,Brasil,RIO GRANDE DO NORTE,MONTE ALEGRE,Caatinga,3.0,0.0,0.75,0,1.4,POINT (-35.36419 -6.10801)
103968,-3.40566,-64.78011,2024-01-17 18:19:00,NOAA-21,Brasil,AMAZONAS,TEFÉ,Amazônia,2.0,0.0,0.00,0,2.3,POINT (-64.78011 -3.40566)
103971,-8.47904,-35.55310,2024-01-17 16:34:00,NOAA-21,Brasil,PERNAMBUCO,BONITO,Mata Atlântica,9.0,0.0,1.00,0,11.2,POINT (-35.5531 -8.47904)
103972,-8.33422,-35.36362,2024-01-17 14:56:00,NOAA-21,Brasil,PERNAMBUCO,PRIMAVERA,Mata Atlântica,3.0,0.0,1.00,0,12.0,POINT (-35.36362 -8.33422)
103973,-6.10410,-35.36360,2024-01-17 03:57:00,NOAA-21,Brasil,RIO GRANDE DO NORTE,MONTE ALEGRE,Caatinga,3.0,0.0,0.73,0,1.4,POINT (-35.3636 -6.1041)


Ocorrências em cada estado:

In [ ]:
num_estado = dados_noa21.estado.value_counts()
num_estado

,count
estado,
PARÁ,447299
MATO GROSSO,392376
AMAZONAS,176362
MARANHÃO,134209
TOCANTINS,125191
MATO GROSSO DO SUL,100596
RONDÔNIA,84201
MINAS GERAIS,73111
PIAUÍ,70205


In [ ]:
estados_completo = gpd.read_file('/content/ufs/BR_UF_2022.shp')
estados_completo['NM_UF'] = estados_completo['NM_UF'].str.upper()
estados_completo.rename(columns={'NM_UF' : 'estado'}, inplace=True)
estados = estados_completo[['estado', 'geometry']].set_index('estado')

Mapa de Calor, Focos Considerando o Risco de Fogo, Durante o Ano de 2024.

In [ ]:
m3 = folium.Map(location=[-23, -50.9253], zoom_start=4)

HeatMap(data=dados_noa21[['latitude', 'longitude', 'risco_fogo']].dropna().values, radius=10, blur=15, max_zoom=6).add_to(m3)
m3

Mapa Coroplético de Focos por Estado.

In [ ]:
m = folium.Map(location=[-23, -50.9253], zoom_start=4)

Choropleth(geo_data = estados.__geo_interface__,
           data = num_estado,
           key_on = 'feature.id',
           fill_color ='OrRd',
           legend_name = 'Focos de Queimadas por Estado em 2024').add_to(m)

m

Importando dados de biomas do IBGE.

In [ ]:
biomas_completo = gpd.read_file('/content/biomas/lm_bioma_250.shp')
biomas_completo.rename(columns={'Bioma': 'bioma'}, inplace = True)
biomas = biomas_completo[['bioma', 'geometry']].set_index('bioma')

biomas = biomas.to_crs(epsg=5880)
biomas['area_km2'] = biomas.area / 1e6

biomas = biomas.to_crs(epsg=4326)

num_bioma = dados_noa21.bioma.value_counts()
num_bioma.head()

Mapa coroplético de Focos de Queimadas por Bioma em 2024.

In [ ]:
m2 = folium.Map(location=[-23, -50.9253], zoom_start=4)

Choropleth(geo_data = biomas.__geo_interface__,
           data = num_bioma,
           key_on = 'feature.id',
           fill_color ='OrRd',
           legend_name = 'Focos de Queimadas por Bioma em 2024').add_to(m2)

GeoJson(
    biomas_completo.__geo_interface__,
    name='Biomas',
    popup=GeoJsonPopup(fields=["bioma"])
).add_to(m2)
m2